## initialize

In [119]:
from ops.imports import *
from ops.imports_ipython import *
from ops import ngs

change directory to the folder containing sgRNA histograms output by the fastq2hist.sh script:

In [4]:
os.chdir('~/data/miniseq/190526')

Set path to the design file (output from original library design)

In [ ]:
f_design = '~/data/libraries/pool5_design.csv'

## find files

In [21]:
search = '*.grep.sg.hist' #extension of sgRNA histogram files
hist_files = natsorted(glob(search))

In [22]:
hist_files

['T3_A01_S2_R1_001.fastq.grep.sg.hist',
 'T3_A02_S3_R1_001.fastq.grep.sg.hist',
 'T3_A03_S4_R1_001.fastq.grep.sg.hist',
 'T3_A04_S5_R1_001.fastq.grep.sg.hist',
 'T3_A05_S6_R1_001.fastq.grep.sg.hist',
 'T3_A06_S7_R1_001.fastq.grep.sg.hist',
 'T3_A07_S8_R1_001.fastq.grep.sg.hist']

## read in histogram files

Create dictionary of {'dataset name':'file path'}

In [74]:
pool_datasets = [f'pool5_{n}' for n in [1,4,5,6,8,9,10]
hist_dict = {dataset:filename for dataset,filename in zip(pool_datasets,hist_files)}

In [76]:
df_hist = ngs.load_sgRNA_hists(hist_dict)

## read in design file

In [77]:
df_design = (
    pd.read_csv(f_design)
            )

In [78]:
df_design = (df_design
             .join((df_design
                    .groupby(['subpool','sgRNA'])
                    .sgRNA.count()
                    .rename('spots_per_oligo')),
                   on = ['subpool','sgRNA']
                  )
             .drop_duplicates(['subpool','sgRNA'])
            )

## identify pools

In [86]:
df_idpool = ngs.identify_pool(df_hist,df_design)

## calculate stats

NGS_Q90_10 here is the "skew ratio"

In [87]:
ngs.calc_stats(df_idpool,df_design)

,,,,NGS_fraction,NGS_Q10,NGS_Q50,NGS_Q90,NGS_Q90_10,NGS_mean,NGS_std,NGS_max,NGS_min,sgRNA_designed,sgRNA_detected,sgRNA_missing
dataset,plate,well,subpool,,,,,,,,,,,,
pool5_1,T3,A01,pool5_1,94.2%,152.0,333.0,589.1,3.875658,354.432828,174.172404,1286,14,1982,1980,2
pool5_10,T3,A07,pool5_10,95.5%,112.0,277.0,481.4,4.298214,291.286424,139.438756,860,6,606,604,2
pool5_4,T3,A02,pool5_4,94.6%,91.0,253.0,513.2,5.639560,283.168301,171.155307,1192,4,1944,1937,7
pool5_5,T3,A03,pool5_5,95.6%,18.0,88.0,281.0,15.611111,124.676808,115.482636,1183,4,7456,7330,126
pool5_6,T3,A04,pool5_6,93.7%,41.0,96.0,169.0,4.121951,101.934862,51.010832,660,4,18239,18146,93
pool5_8,T3,A05,pool5_8,87.5%,183.6,912.0,3541.6,19.289760,1886.000000,3193.546096,10720,117,10,10,0
pool5_9,T3,A06,pool5_9,97.0%,79.9,217.0,368.4,4.610763,233.460000,131.398801,695,18,100,100,0
